# U1 annommalies paralelization 

**by: Nicole Rivera**

This code runs a proposed paralelization to find U1 annommalies described in [Costa *et.al.*](https://doi.org/10.48550/arXiv.1905.13729). 

In [1]:
from anomalies import anomaly
import numpy as np
import multiprocessing
import time 

## Anomalies pakage

First we implement paralelization using [anomalies 0.2.4](https://pypi.org/project/anomalies/) pip package

In [2]:
def U1_sol(n):
    """n size solution calculator"""
    
    # choose m according to parity
    if n%2 == 0 and n > 2:
        m = int(n/2 - 1)
    elif n%2 != 0 and n >= 5:
        m = int((n - 3)/2)
    
    # create k,l of size m
    k = np.random.randint(-5, 5, size=m)
    l = np.random.randint(-6, 6, size=m)
    
    # get U1 solution of size n
    U1_ns = anomaly.free(l, k)
    U1_s = anomaly.free.simplified   
    
    return U1_s

def U1_quiral_solutions(n0):
    """ U1 quiral solutions selection function """
    
    # set acceptance variable
    ac = 0
    while ac == 0: # repeat while acceptance is cero
        
        z = U1_sol(n0) # proposed solution
        
        # discard if any value is greater than 30
        if np.any(np.abs(z)>30) or np.any(np.abs(z) == 0):
            #print("mayor que 30")
            ac = 0
            continue
        
        # vector-like comprobation
        uniq_abs = np.unique(np.abs(z)).shape[0]
        uniq_all = np.unique(z).shape[0]
        
        if uniq_abs == uniq_all:
            #print("quiral solution")
            ac = 1
            break 
        else:
            #print("vector-like solution")
            ac = 0
            
    return z

In [3]:
t0 = time.perf_counter()
z_t = U1_quiral_solutions(6)
tf = time.perf_counter()
print(f"Program finished in {tf-t0} seconds")
print(z_t)

Program finished in 0.002628275000006397 seconds
[  1   4   5 -12 -15  17]


In [4]:
### --- multiprocessing --- #

if __name__ == "__main__":
    pool = multiprocessing.Pool(3)
    start_time = time.perf_counter()
    processes = [pool.apply_async(U1_quiral_solutions, args=(n,)) for n in range(5, 8)]
    result = [p.get() for p in processes]
    finish_time = time.perf_counter()
    print(f"Program finished in {finish_time-start_time} seconds")
    print(result)

/home/nicole/.local/lib/python3.8/site-packages/anomalies/anomaly.py:49: RuntimeWarning: invalid value encountered in divide
  self.simplified=(zz/self.gcd).astype(int)
/home/nicole/.local/lib/python3.8/site-packages/anomalies/anomaly.py:49: RuntimeWarning: invalid value encountered in divide
  self.simplified=(zz/self.gcd).astype(int)


Program finished in 0.0049578290000908964 seconds
[array([-9223372036854775808, -9223372036854775808, -9223372036854775808,
       -9223372036854775808]), array([-9223372036854775808, -9223372036854775808, -9223372036854775808,
       -9223372036854775808, -9223372036854775808, -9223372036854775808]), array([-9223372036854775808, -9223372036854775808, -9223372036854775808,
       -9223372036854775808, -9223372036854775808, -9223372036854775808])]


/home/nicole/.local/lib/python3.8/site-packages/anomalies/anomaly.py:49: RuntimeWarning: invalid value encountered in divide
  self.simplified=(zz/self.gcd).astype(int)


## Own code

Then we implement the paralelization using only our code. This is  made because there is a possible implementation error using anomalies library.

In [2]:
def merge_op(x, y):
    """ merge operation for z with specified x, y"""
    return np.sum( x*(y**2) )*x - np.sum( (x**2) * y )*y 

def even(n):
    """ even sized U1 solution calculator"""
    
    m = int(n/2 - 1) # k, l size 

    k = np.random.randint(1, 10, size=m)
    l = np.random.randint(1, 10, size=m)
    
    # generate plus and minus vector 
    vp = np.concatenate(([l[0]], k, [-l[0]], -k))
    vm = np.concatenate(([0,0], l, -l))
    
    # calculate z
    z = merge_op(vp, vm)
    
    return z

def odd(n):
    """ odd sized U1 solution calculator"""
    
    m = int((n - 3)/2) # k, l size
    
    k = np.random.randint(1, 10, size=m+1)
    l = np.random.randint(1, 10, size=m)
    
    # generate plus and minus vector 
    up = np.concatenate(([0], k, -k))
    um = np.concatenate((l, [k[0], 0], -l, [-k[0]]))
    
    # calculate z
    z = merge_op(up, um)
    
    return z

def no_vectorlike(z):
    """ classifies if z is vector-like or quiral solution """
    
    # gets unique values of z and its absolute value elemets
    uniq_abs = np.unique(np.abs(z)).shape[0]
    uniq_all = np.unique(z).shape[0]
    
    if uniq_abs == uniq_all:
        # if unique values are equal to absolute unique values the z is quiral
        return 1
    else:
        # otherwise is vector-like
        return 0

def prueba_U1(q):
    """ proof operations of U1 group """
    
    a3 = np.sum(q**3)
    a1 = np.sum(q)
    
    return (a3, a1)

### ---- Funcion principal ----###
def joint(n0):
    """ main function to calculate quiral U1 solutions """
    
    rs = 0 # aceptance variable - 0 is no-quiral, 1 is quiral
    
    while rs == 0: 
        
        # gets z according to case
        if n0%2 == 0 and n0 > 2:
            zf = even(n0)
        elif n0%2 != 0 and n0 >= 5:
            zf = odd(n0)
        else:
            print("ingrese entero positivo válido")
            rs = 2 
            break
            
        div = np.gcd.reduce(zf) # finds z greater common divisor
    
        rs = no_vectorlike(zf) # evaluates if z is vector-like
        
        # evaluates other conditions over z
        if rs  == 1:
            zn = zf/div # reduced z by its gcd
            if np.any(np.abs(zn)>30) or np.any(np.abs(zn) == 0):
                # all elements must be non zero and minor than 30
                rs = 0
                continue
            else:
                #return (zn, div) #use this if you also need gcd information
                return zn # use this if you only need z 
                break

In [3]:
t0 = time.perf_counter()
z_n = joint(6)
tf = time.perf_counter()
print(f"Program finished in {tf-t0} seconds")
print(z_n)

Program finished in 0.0077715309998893645 seconds
[-12. -15. -20.  17.  11.  19.]


In [7]:
# --- size of solution --- #
n_values = np.full(10000, 5)

In [8]:
# --- multiprocessing --- #

if __name__ == "__main__":
    pool = multiprocessing.Pool(4)
    start_time = time.perf_counter()
    result = pool.map(joint, n_values)
    finish_time = time.perf_counter()
    print(f"Program finished in {finish_time-start_time} seconds")
    
    for z in np.unique(result, axis=0):
        print(z)

Program finished in 1.165443759000027 seconds
[-25. -14.   9.   4.  26.]
[-23.   2.  18. -25.  28.]
[-22.  -6.  12.  -5.  21.]
[-22.   9.   7. -20.  26.]
[-21. -12.   5.   6.  22.]
[-21. -12.   6.   5.  22.]
[-18. -28.  -2.  25.  23.]
[-18.  -2.  23. -28.  25.]
[-17.   8.   7. -25.  27.]
[-14. -20.  -1.  18.  17.]
[-14.   5.   8. -26.  27.]
[-12.   6.  22. -21.   5.]
[-10.  -2.   7.  -4.   9.]
[-9.  4.  2. -7. 10.]
[ -9.  14.  25. -26.  -4.]
[ -8.  -5.  14. -27.  26.]
[ -7. -27.  -8.  25.  17.]
[ -7.  -9.  22. -26.  20.]
[ -7.  -8.  17. -27.  25.]
[-7.  1.  5. -8.  9.]
[-7.  2.  4. -9. 10.]
[-7.  2. 10. -9.  4.]
[ -6.  12.  21. -22.  -5.]
[-5. -9. -1.  8.  7.]
[-5. -1.  7. -9.  8.]
[ -5.  12.  21. -22.  -6.]
[ -4. -10.  -2.   9.   7.]
[ -4.  -2.   7. -10.   9.]
[ -2.  -4.   9. -10.   7.]
[ 1.  9.  5. -7. -8.]
[  1.  20.  14. -17. -18.]
[ 2. 10.  4. -7. -9.]
[  2.  28.  18. -23. -25.]
[ 4. 10. -9.  2. -7.]
[ 5. -8. -7.  9.  1.]
[ 5. -7. -8.  9.  1.]
[ 5.  9. -8.  1. -7.]
[ 5.  9. -7.  1